# Custom Training StyleGan2-ADA

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov  7 06:37:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
if os.path.isdir("/content/drive/MyDrive/AIB_04_Stylebot/colab-sg2-ada"):
    %cd "/content/drive/MyDrive/AIB_04_Stylebot/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/MyDrive/AIB_04_Stylebot"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/.shortcut-targets-by-id/1zQGyqDeW63FEr3sXvFz23SVnqvoGEBUd/AIB_04_Stylebot/colab-sg2-ada/stylegan2-ada


In [4]:
# 필요한 라이브러리
import cv2
from os import listdir
from os.path import isfile, join
from google.colab import files
from PIL import Image, ImageDraw
import numpy as np
from tqdm import tqdm
import imageio
import os
from pathlib import Path
import glob
import matplotlib.pyplot as plt

In [ ]:
# 1024 * 1024 size

result_path = '/content/drive/MyDrive/AIB_04_Stylebot/selected_crop'

image_files = Path(result_path)
image_files = list(image_files.glob(r'*.jpg'))

for name in image_files[:]:
  img = cv2.imread(f'{name}')
  img = cv2.resize(img, (1024,1024))
  # 전처리 이미지 저장
  cv2.imwrite(os.path.join(path , 'waka.jpg'), img)
  # img.shape 변경 확인
  print(img.shape)

## Convert dataset to .tfrecords

##tfrecords
TFRecord는 데이터 세트의 포맷의 하나로 TFRecord형식은 바이너리 코드의 시리즈를 저장하기 위한 단순한 형식

==> 대규모 데이터를 효율적으로 학습할 수 있게 된다

==> StyleGAN2-ADA가 읽을 수 있게 TFRecord로 변환

In [9]:
# 파라미터 설명
!python dataset_tool.py --help

usage: dataset_tool.py [-h]
                       {info,display,extract,compare,create_mnist,create_mnistrgb,create_cifar10,create_cifar100,create_svhn,create_lsun,create_lsun_wide,create_celeba,create_from_images,create_from_images_raw,create_from_image_folders,create_from_image_folders_raw,create_from_images_with_labels,create_from_hdf5,convert_to_hdf5,hdf5_from_images,unpack,pack,extract_brecahad_crops}
                       ...

Tool for creating multi-resolution TFRecords datasets for StyleGAN and ProGAN.

positional arguments:
  {info,display,extract,compare,create_mnist,create_mnistrgb,create_cifar10,create_cifar100,create_svhn,create_lsun,create_lsun_wide,create_celeba,create_from_images,create_from_images_raw,create_from_image_folders,create_from_image_folders_raw,create_from_images_with_labels,create_from_hdf5,convert_to_hdf5,hdf5_from_images,unpack,pack,extract_brecahad_crops}
    info                Display general info about dataset.
    display             Display image

In [ ]:
!python dataset_tool.py create_from_images /content/drive/MyDrive/AIB_04_Stylebot/colab-sg2-ada/stylegan2-ada/datasets/{'1024image'} {result_path}

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Loading images from "/content/drive/MyDrive/AIB_04_Stylebot/selected_crop"
Creating dataset "/content/drive/MyDrive/AIB_04_Stylebot/colab-sg2-ada/stylegan2-ada/datasets/1024image"
dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 55 images.


## Train a custom model


In [ ]:
# 파라미터 설명
!python train.py --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "1024image"
#how often should the model generate samples and a .pkl file
snapshot_count = 4
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bg"
'''
augpipe_specs = {
        'blit':     dict(xflip=1, rotate90=1, xint=1),
        'geom':     dict(scale=1, rotate=1, aniso=1, xfrac=1),
        'color':    dict(brightness=1, contrast=1, lumaflip=1, hue=1, saturation=1),
        'filter':   dict(imgfilter=1),
        'noise':    dict(noise=1),
        'cutout':   dict(cutout=1),
        'bg':       dict(xflip=1, rotate90=1, xint=1, scale=1, rotate=1, aniso=1, xfrac=1),
        'bgc':      dict(xflip=1, rotate90=1, xint=1, scale=1, rotate=1, aniso=1, xfrac=1, brightness=1, contrast=1, lumaflip=1, hue=1, saturation=1),
        'bgcf':     dict(xflip=1, rotate90=1, xint=1, scale=1, rotate=1, aniso=1, xfrac=1, brightness=1, contrast=1, lumaflip=1, hue=1, saturation=1, imgfilter=1),
        'bgcfn':    dict(xflip=1, rotate90=1, xint=1, scale=1, rotate=1, aniso=1, xfrac=1, brightness=1, contrast=1, lumaflip=1, hue=1, saturation=1, imgfilter=1, noise=1),
        'bgcfnc':   dict(xflip=1, rotate90=1, xint=1, scale=1, rotate=1, aniso=1, xfrac=1, brightness=1, contrast=1, lumaflip=1, hue=1, saturation=1, imgfilter=1, noise=1, cutout=1),
    }

# Discriminator augmentation. = Generator는 real data의 분포를 학습하기 때문에
aug        = None, # Augmentation mode: 'ada' (default), 'noaug', 'fixed', 'adarv'
p          = None, # Specify p for 'fixed' (required): <float>
target     = None, # Override ADA target for 'ada' and 'adarv': <float>, default = depends on aug
augpipe    = None, # Augmentation pipeline: 'blit', 'geom', 'color', 'filter', 'noise', 'cutout', 'bg', 'bgc' (default), ..., 'bgcfnc'

# Transfer learning.
resume     = None, # Load previous network: 'noresume' (default), 'ffhq256', 'ffhq512', 'ffhq1024', 'celebahq256', 'lsundog256', <file>, <url>
'''
# Transfer learning.
resume_from = "ffhq1024" 

#kimg = batch?
!python train.py --outdir ./results --snap={snapshot_count} --cfg=stylegan2 --kimg=100 --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from}

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
tcmalloc: large alloc 4294967296 bytes == 0x558350a92000 @  0x7f97f48ce001 0x7f97f1ad154f 0x7f97f1b21b58 0x7f97f1b25b17 0x7f97f1bc4203 0x558349a22544 0x558349a22240 0x558349a96627 0x558349a90ced 0x558349a2448c 0x558349a65159 0x558349a620a4 0x558349a22d49 0x558349a9694f 0x558349a909ee 0x558349962e2b 0x558349a92fe4 0x558349a909ee 0x558349962e2b 0x558349a92fe4 0x558349a90ced 0x558349962e2b 0x558349a92fe4 0x558349a23afa 0x558349a91915 0x558349a909ee 0x558349a906f3 0x558349b5a4c2 0x558349b5a83d 0x558349b5a6e6 0x558349b32163
tcmalloc: large alloc 4294967296 bytes == 0x558450a92000 @  0x7f97f48cc1e7 0x7f97f1ad146e 0x7f97f1b21c7b 0x7f97f1b2235f 0x7f97f1bc4103 0x558349a22544 0x558349a22240 0x558349a96627 0x558349a909ee 0x558349a23bda 0x558349a92737 0x558349a909ee 0x558349a23bda 0x558349a92737 0x558349a909ee

# **Generate**

In [6]:
!pip install opensimplex

In [10]:
!python generate.py generate-images --help

usage: generate.py generate-images [-h] --network NETWORK_PKL --seeds SEEDS
                                   [--trunc TRUNCATION_PSI]
                                   [--class CLASS_IDX] [--create-grid]
                                   [--outdir DIR] [--save_vector] [--fixnoise]
                                   [--jpg_quality JPG_QUALITY]

optional arguments:
  -h, --help            show this help message and exit
  --network NETWORK_PKL
                        Network pickle filename
  --seeds SEEDS         List of random seeds
  --trunc TRUNCATION_PSI
                        Truncation psi (default: 0.5)
  --class CLASS_IDX     Class label (default: unconditional)
  --create-grid         Add flag to save the generated images in a grid
  --outdir DIR          Root directory for run results (default: out)
  --save_vector         also save vector in .npy format
  --fixnoise            generate images using fixed noise (more accurate for
                        interpolations)
  

In [ ]:
network = '/content/drive/MyDrive/AIB_04_Stylebot/network-snapshot-000200.pkl'

#--create-grid
'''
Seed must be between 0 and 2**32 - 1

for seed_idx, seed in enumerate(seeds):
    print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
    rnd = np.random.RandomState(seed)
    z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
    tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
    images = Gs.run(z, label, **Gs_kwargs) # [minibatch, height, width, channel]
    PIL.Image.fromarray(images[0], 'RGB').save(f'{outdir}/seed{seed:04d}.png')
'''

!python generate.py generate-images --outdir='/content/image' --trunc=0 --seeds=0-555 --network={network} 

In [ ]:
!python style_mixing.py --help

usage: style_mixing.py [-h] --network NETWORK_PKL --rows ROW_SEEDS --cols
                       COL_SEEDS [--styles COL_STYLES]
                       [--trunc TRUNCATION_PSI] --outdir DIR

Generate style mixing image matrix using pretrained network pickle.

optional arguments:
  -h, --help            show this help message and exit
  --network NETWORK_PKL
                        Network pickle filename
  --rows ROW_SEEDS      Random seeds to use for image rows
  --cols COL_SEEDS      Random seeds to use for image columns
  --styles COL_STYLES   Style layer range (default: 0-6)
  --trunc TRUNCATION_PSI
                        Truncation psi (default: 0.5)
  --outdir DIR          Where to save the output images

examples:

  python style_mixing.py --outdir=out --trunc=1 --rows=85,100,75,458,1500 --cols=55,821,1789,293 \
      --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/metfaces.pkl


In [ ]:
!python style_mixing.py --outdir='/content/drive/MyDrive/AIB_04_Stylebot/colab-sg2-ada/style_mixing/style=0-3 1024' --trunc=0.5 --rows=4585 --cols=3333,1548,5184,6954,4878  --network={network} --styles=0-3

Loading networks from "/content/drive/MyDrive/AIB_04_Stylebot/network-snapshot-000100.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating W vectors...
Generating images...
Generating style-mixed images...
Saving images...
Saving image grid...
